# Unit Square

Here we demonstrate how to plot different FEniCSx object for a unit square geometry

First we make the necessary imports

In [ ]:
import dolfinx
import numpy as np
from mpi4py import MPI
import ufl
from fenicsx_plotly import plot

First we create the unit square geometry

In [ ]:
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 3, 3)

Now we can plot the mesh. By default it will plot the mesh in wireframe

In [ ]:
plot(mesh)

We can also turn off wireframe mode

In [ ]:
plot(mesh, wireframe=False)

Now we can try to create a function space and plot the degrees of freedom

In [ ]:
V = dolfinx.fem.FunctionSpace(mesh, ("P",  3))
plot(V)

We can also plot a function in this function space

In [ ]:
p = dolfinx.fem.Function(V)
p.interpolate(lambda x: np.sin(x[0]))
plot(p, scatter=True, wireframe=False, show_grid=True)

We can also create a vector function space

In [ ]:
W = dolfinx.fem.FunctionSpace(mesh, ufl.VectorElement("P", mesh.ufl_cell(), 2))
plot(W)

and create a function in this function space

In [ ]:
u = dolfinx.fem.Function(W)
# Just create create some non-trivial function
x = ufl.SpatialCoordinate(mesh)
expr = dolfinx.fem.Expression(
    ufl.as_vector((1 + x[0], x[1])), W.element.interpolation_points()
)

u.interpolate(expr)

plot(u, size=1)
plot(u, normalize=True, size=1)

We can also plot the different components

In [ ]:
for component in ["magnitude", "x", "y"]:
    plot(u, component=component)

And we can plot mesh tags

In [ ]:
locator = lambda x: x[0] < 0.5
entities = dolfinx.mesh.locate_entities(mesh, 2, locator)
marker = 1
values = np.full_like(entities, marker)

facet_tags = dolfinx.mesh.meshtags(mesh, 2, entities, values)
plot(facet_tags, show_grid=True)

And dirichlet BC (currently only scalar values)

In [ ]:
boundary = lambda x: np.isclose(x[0], 0)

el = ufl.FiniteElement("P", mesh.ufl_cell(), 2)
V = dolfinx.fem.FunctionSpace(mesh, el)

dofs_D = dolfinx.fem.locate_dofs_geometrical(V, boundary)
u_exact = lambda x: 1 + x[1]

u_bc = dolfinx.fem.Function(V)
u_bc.interpolate(u_exact)
bc = dolfinx.fem.dirichletbc(u_bc, dofs_D)
plot(bc, show_grid=True)